In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'NanumGothic'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
major = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240317상장주력_stand.csv', index_col=0)
major_test = pd.read_csv('./drive/MyDrive/공유문서함/상장csv/240318상장주력test_stand.csv', index_col=0)

In [ ]:
# major.drop(['회사명',
#   '회계년도',
#   '설립일',
#   '상장일',
#   '상장폐지일',
#   '상장협 산업분류(소분류)_x',
#   '상장협 산업분류(중분류)_x',
#   '회계년',
#   'Unnamed: 0'], axis=1, inplace=True)

##### train_df, test_df 설정

In [14]:
train_df = major[['CASH FLOW 대 매출액비율',
                    '매출액순이익률',
                    '1주당매출액',
                    '자기자본순이익률',
                    '적립금비율(재정비율)',
                    '자본',
                    '재고자산 대 유동자산비율',
                    '매출채권회전기간',
                    '비유동부채비율',
                    '순운전자본회전률',
                    '유보율',
                    'CASH FLOW 대 차입금비율',
                    '유동부채비율',
                    '부실']]

In [15]:
test_df = major_test[['CASH FLOW 대 매출액비율',
                    '매출액순이익률',
                    '1주당매출액',
                    '자기자본순이익률',
                    '적립금비율(재정비율)',
                    '자본',
                    '재고자산 대 유동자산비율',
                    '매출채권회전기간',
                    '비유동부채비율',
                    '순운전자본회전률',
                    '유보율',
                    'CASH FLOW 대 차입금비율',
                    '유동부채비율',
                    '부실']]

In [16]:
x_train = train_df.drop("부실",axis=1)
y_train = train_df["부실"]
x_test =test_df.drop("부실",axis=1)
y_test = test_df["부실"]

In [ ]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    pr_score = average_precision_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
y_train.value_counts()

0.0    4669
1.0      88
Name: 부실, dtype: int64

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# RandomUnderSampler 객체 생성
under = RandomUnderSampler(random_state=4, sampling_strategy=0.3)

# RandomUnderSampler를 사용하여 샘플링된 특징 및 타겟 데이터 생성
x_resampled, y_resampled = under.fit_resample(x_train, y_train)

y_resampled.value_counts()

0.0    293
1.0     88
Name: 부실, dtype: int64

In [ ]:
# from imblearn.over_sampling import RandomOverSampler

# x_resampled2, y_resampled2 = RandomOverSampler(random_state=4, sampling_strategy=1).fit_resample(x_resampled, y_resampled )
# print(x_train.shape)
# y_resampled2.value_counts()

(3485, 13)


0.0    540
1.0    540
Name: 부실, dtype: int64

#### 1. Tabnet

In [ ]:
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 901.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier

# TabNet 모델 생성
tabnet_clf = TabNetClassifier()

# DataFrame을 NumPy 배열로 변환
x_train_np = x_resampled.values
y_train_np = y_resampled.values
x_test_np = x_test.values

# 모델 훈련
tabnet_clf.fit(x_train_np, y_train_np)

# 테스트 데이터에 대한 예측
tabnet_pred = tabnet_clf.predict(x_test_np)
tabnet_pred_proba = tabnet_clf.predict_proba(x_test_np)

epoch 0  | loss: 0.0     |  0:00:00s
epoch 1  | loss: 0.0     |  0:00:00s
epoch 2  | loss: 0.0     |  0:00:00s
epoch 3  | loss: 0.0     |  0:00:00s
epoch 4  | loss: 0.0     |  0:00:00s
epoch 5  | loss: 0.0     |  0:00:00s
epoch 6  | loss: 0.0     |  0:00:00s
epoch 7  | loss: 0.0     |  0:00:00s
epoch 8  | loss: 0.0     |  0:00:00s
epoch 9  | loss: 0.0     |  0:00:00s
epoch 10 | loss: 0.0     |  0:00:00s
epoch 11 | loss: 0.0     |  0:00:00s
epoch 12 | loss: 0.0     |  0:00:00s
epoch 13 | loss: 0.0     |  0:00:00s
epoch 14 | loss: 0.0     |  0:00:00s
epoch 15 | loss: 0.0     |  0:00:00s
epoch 16 | loss: 0.0     |  0:00:00s
epoch 17 | loss: 0.0     |  0:00:00s
epoch 18 | loss: 0.0     |  0:00:00s
epoch 19 | loss: 0.0     |  0:00:00s
epoch 20 | loss: 0.0     |  0:00:00s
epoch 21 | loss: 0.0     |  0:00:00s
epoch 22 | loss: 0.0     |  0:00:00s
epoch 23 | loss: 0.0     |  0:00:00s
epoch 24 | loss: 0.0     |  0:00:00s
epoch 25 | loss: 0.0     |  0:00:00s
epoch 26 | loss: 0.0     |  0:00:00s
e

In [ ]:
# 성능 평가
print("TabNet")
get_clf_eval(y_test, tabnet_pred)

TabNet
오차행렬
[[1731    0]
 [  19    0]]
정확도: 0.9891, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000


In [ ]:
# from pytorch_tabnet.tab_model import TabNetClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import make_scorer, accuracy_score

# # TabNet 모델 정의
# tabnet_clf = TabNetClassifier()

# # 그리드 서치를 위한 하이퍼파라미터 그리드 설정
# param_grid = {
#     'n_d': [8, 16, 32],  # Decision 단위의 피처 개수
#     'n_a': [8, 16, 32],  # Attention 단위의 피처 개수
#     'n_steps': [3, 5, 10],  # 역전파 단계 수
#     'gamma': [1.0, 1.5, 2.0],  # 역전파 시 감마 값
#     'lambda_sparse': [0.0001, 0.001, 0.01],  # 희소 손실 정규화 파라미터
# }

# # 그리드 서치 수행
# grid_search = GridSearchCV(tabnet_clf, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(x_train_np, y_train_np)

# # 최적의 모델과 하이퍼파라미터 가져오기
# best_tabnet_clf = grid_search.best_estimator_
# best_params = grid_search.best_params_

# # 테스트 데이터에 대한 예측 수행
# tabnet_pred = best_tabnet_clf.predict(x_test)
# tabnet_pred_proba = best_tabnet_clf.predict_proba(x_test)

In [ ]:
# 분류 임계값 조정
threshold = 0.3  # 임계값 설정
tabnet_pred_adjusted = (tabnet_pred_proba[:, 1] >= threshold).astype(int)

# 임곗값 조정 후 성능 평가
print("TabNet")
get_clf_eval(y_test, tabnet_pred)

TabNet
오차행렬
[[1731    0]
 [  19    0]]
정확도: 0.9891, 정밀도 : 0.0000, 재현율:0.0000,F1 스코어:0.0000


#### 2. LSTM

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(y_true, y_pred):
    # 정확도 (Accuracy)
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)

    # 정밀도 (Precision)
    precision = precision_score(y_true, y_pred)
    print("Precision:", precision)

    # 재현율 (Recall)
    recall = recall_score(y_true, y_pred)
    print("Recall:", recall)

    # F1 스코어 (F1 Score)
    f1 = f1_score(y_true, y_pred)
    print("F1 Score:", f1)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_true, y_pred)
        print("AUC-ROC:", auc_roc)
    except ValueError:
        print("AUC-ROC cannot be calculated for multiclass classification.")

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Model Building
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(x_resampled.shape[1],1)))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Model Compilation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
model.fit(x_resampled, y_resampled, epochs=100, batch_size=64)

# 모델 평가
y_pred = model.predict(x_test)
predictions_binary = (y_pred > 0.5).astype(int)

evaluate(y_test, predictions_binary)

Epoch 1/100
6/6 [==============================] - 2s 47ms/step - loss: 0.6781 - accuracy: 0.7717
Epoch 2/100
6/6 [==============================] - 0s 39ms/step - loss: 0.6449 - accuracy: 0.7769
Epoch 3/100
6/6 [==============================] - 0s 42ms/step - loss: 0.5766 - accuracy: 0.7769
Epoch 4/100
6/6 [==============================] - 0s 38ms/step - loss: 0.4173 - accuracy: 0.8425
Epoch 5/100
6/6 [==============================] - 0s 27ms/step - loss: 0.2834 - accuracy: 0.8819
Epoch 6/100
6/6 [==============================] - 0s 27ms/step - loss: 0.2620 - accuracy: 0.8871
Epoch 7/100
6/6 [==============================] - 0s 25ms/step - loss: 0.2785 - accuracy: 0.9003
Epoch 8/100
6/6 [==============================] - 0s 24ms/step - loss: 0.2553 - accuracy: 0.8976
Epoch 9/100
6/6 [==============================] - 0s 24ms/step - loss: 0.2499 - accuracy: 0.9003
Epoch 10/100
6/6 [==============================] - 0s 26ms/step - loss: 0.2409 - accuracy: 0.8871
Epoch 11/100
6/6 [=

In [ ]:
# 임계값에 따라 예측값 조정
threshold = 0.3 # 기본 임계값

predictions_binary = (y_pred > threshold).astype(int)
evaluate(y_test, predictions_binary)

Accuracy: 0.94
Precision: 0.10185185185185185
Recall: 0.5789473684210527
F1 Score: 0.17322834645669288
AUC-ROC: 0.7614551977864938


#### 3. CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# 모델 생성
model = Sequential()
model.add(Conv1D(128, 3, activation='relu', input_shape=(x_resampled.shape[1],1)))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델 학습
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_resampled, y_resampled, epochs=100, batch_size=64)

# 모델 평가
y_pred = model.predict(x_test)

predictions_binary = (y_pred > 0.5).astype(int)
evaluate(y_test, predictions_binary)

Epoch 1/100
6/6 [==============================] - 2s 14ms/step - loss: 0.5947 - accuracy: 0.8373
Epoch 2/100
6/6 [==============================] - 0s 16ms/step - loss: 0.4329 - accuracy: 0.8898
Epoch 3/100
6/6 [==============================] - 0s 17ms/step - loss: 0.3496 - accuracy: 0.8845
Epoch 4/100
6/6 [==============================] - 0s 13ms/step - loss: 0.3094 - accuracy: 0.8898
Epoch 5/100
6/6 [==============================] - 0s 14ms/step - loss: 0.2933 - accuracy: 0.8871
Epoch 6/100
6/6 [==============================] - 0s 14ms/step - loss: 0.2748 - accuracy: 0.8871
Epoch 7/100
6/6 [==============================] - 0s 10ms/step - loss: 0.2690 - accuracy: 0.8976
Epoch 8/100
6/6 [==============================] - 0s 13ms/step - loss: 0.2596 - accuracy: 0.9029
Epoch 9/100
6/6 [==============================] - 0s 14ms/step - loss: 0.2476 - accuracy: 0.9055
Epoch 10/100
6/6 [==============================] - 0s 10ms/step - loss: 0.2479 - accuracy: 0.9081
Epoch 11/100
6/6 [=

In [ ]:
# 임계값에 따라 예측값 조정
threshold = 0.2 # 기본 임계값

predictions_binary = (y_pred > threshold).astype(int)
evaluate(y_test, predictions_binary)

Accuracy: 0.9497142857142857
Precision: 0.12903225806451613
Recall: 0.631578947368421
F1 Score: 0.21428571428571427
AUC-ROC: 0.7923925932682659


#### 교차검증

In [9]:
fix_list = ['회사명',
            '회계년도',
            '설립일',
            '상장일',
            '상장폐지일',
            '상장협 산업분류(소분류)_x',
            '상장협 산업분류(중분류)_x',
            '회계년',
            'Unnamed: 0'
            ]

def label_split(df):
  feature = df.drop(fix_list, axis=1)
  label = df['부실']
  return feature, label

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def evaluate(y_true, y_pred):
    # 정확도 (Accuracy)
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)

    # 정밀도 (Precision)
    precision = precision_score(y_true, y_pred)
    print("Precision:", precision)

    # 재현율 (Recall)
    recall = recall_score(y_true, y_pred)
    print("Recall:", recall)

    # F1 스코어 (F1 Score)
    f1 = f1_score(y_true, y_pred)
    print("F1 Score:", f1)

    # AUC-ROC
    try:
        auc_roc = roc_auc_score(y_true, y_pred)
        print("AUC-ROC:", auc_roc)
    except ValueError:
        print("AUC-ROC cannot be calculated for multiclass classification.")

In [7]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.4 MB/s eta 0:00:00


In [11]:
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


def modeling_cv(train, test):
    print(train['부실'].value_counts())

    feature, label = label_split(train)
    test_feature, test_label = label_split(test)

    feature, label = label_split(train)
    test_feature, test_label = label_split(test)

    print(label.value_counts())
    print(test_label.value_counts())

    models = [
        # ("Logistic Regression", LogisticRegression()),
        # ("KNN", KNeighborsClassifier(n_neighbors=5)),
        # ("Random Forest", RandomForestClassifier(n_estimators=100, random_state=42)),
        # ("XGBoost", xgb.XGBClassifier()),
        # ("LightGBM", lgb.LGBMClassifier()),
        ("CatBoost", CatBoostClassifier()),
        ("SVM (Linear Kernel)", SVC(kernel='linear', probability=True)),
    ]

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Adjust n_splits as needed

    print("Stratified K-Fold Cross-Validation Performance:")
    for name, model in models:
        print(f"\n---{name}---")
        for train_index, val_index in cv.split(feature, label):
            X_train, X_val = feature.iloc[train_index], feature.iloc[val_index]
            y_train, y_val = label.iloc[train_index], label.iloc[val_index]

            undersampler = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
            X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

            print(y_train_resampled.value_counts())

            model.fit(X_train_resampled, y_train_resampled)  # Train the model on the training fold

            # Use trained model to make predictions on the validation fold
            if hasattr(model, "predict_proba"):
                y_pred_prob = model.predict_proba(X_val)[:, 1]
                y_pred = (y_pred_prob >= 0.5).astype(int)  # Adjust threshold as needed
            else:
                y_pred = model.predict(X_val)

            print(f"---{name}---")
            evaluate(y_val, y_pred)
            print()

In [12]:
modeling_cv(major, major_test)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
103:	learn: 0.0429778	total: 2.23s	remaining: 19.2s
104:	learn: 0.0423785	total: 2.25s	remaining: 19.2s
105:	learn: 0.0415838	total: 2.27s	remaining: 19.2s
106:	learn: 0.0408559	total: 2.29s	remaining: 19.1s
107:	learn: 0.0401641	total: 2.31s	remaining: 19.1s
108:	learn: 0.0394239	total: 2.33s	remaining: 19s
109:	learn: 0.0386531	total: 2.35s	remaining: 19s
110:	learn: 0.0378486	total: 2.37s	remaining: 19s
111:	learn: 0.0365865	total: 2.37s	remaining: 18.8s
112:	learn: 0.0359173	total: 2.39s	remaining: 18.8s
113:	learn: 0.0351814	total: 2.41s	remaining: 18.7s
114:	learn: 0.0346389	total: 2.43s	remaining: 18.7s
115:	learn: 0.0339951	total: 2.45s	remaining: 18.7s
116:	learn: 0.0332055	total: 2.46s	remaining: 18.5s
117:	learn: 0.0327574	total: 2.48s	remaining: 18.5s
118:	learn: 0.0321290	total: 2.49s	remaining: 18.5s
119:	learn: 0.0316892	total: 2.51s	remaining: 18.4s
120:	learn: 0.0311504	total: 2.53s	remaining: 18.4s
121:	learn: 0.0304458	total: 2.53s

In [26]:
def calculate_f1_score(recall, precision):
    """
    Calculate F1-score using recall and precision.

    Parameters:
    - recall: Recall score.
    - precision: Precision score.

    Returns:
    - f1: F1-score.
    """
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [31]:
# F1-score 계산
f1 = calculate_f1_score(0.10, 0.07)
print("F1-score:", f1)

F1-score: 0.0823529411764706


#### 교차검증

In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler

# STRAtified k-fold를 위한 설정
k_folds = 5  # k를 설정합니다.
stratified_kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# 교차 검증 수행
accuracies = []  # 각 fold의 정확도를 저장할 리스트

for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(x_train, y_train)):
    print(f"Fold {fold+1}/{k_folds}")

    # Fold에 따라 데이터 분할
    X_train_fold, X_val_fold = x_train.iloc[train_idx], x_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # undersampling 수행
    rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train_fold, y_train_fold)

    X_train_resampled = X_train_resampled.values.reshape(X_train_resampled.shape[0], 1, X_train_resampled.shape[1])
    y_train_resampled = y_train_resampled.values

    # 모델 생성
    model = Sequential()
    model.add(LSTM(128, activation='relu', input_shape=(X_train_resampled.shape[1:])))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # 모델 학습
    model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=64, verbose=0)  # verbose=0으로 설정하여 학습 과정 출력 생략

    # 검증 데이터에 대한 예측 및 평가
    X_val_fold = X_val_fold.values.reshape(X_val_fold.shape[0], 1, X_val_fold.shape[1])
    y_pred = model.predict(X_val_fold)

    # 이진 분류로 변환
    y_pred_binary = (y_pred > 0.3).astype(int)

    evaluate(y_val_fold,y_pred_binary)

Fold 1/5
30/30 [==============================] - 0s 2ms/step
Accuracy: 0.9464285714285714
Precision: 0.2
Recall: 0.6111111111111112
F1 Score: 0.30136986301369867
AUC-ROC: 0.7820009517011659
Fold 2/5
30/30 [==============================] - 0s 1ms/step
Accuracy: 0.9548319327731093
Precision: 0.20930232558139536
Recall: 0.5
F1 Score: 0.29508196721311475
AUC-ROC: 0.731798715203426
Fold 3/5
30/30 [==============================] - 0s 2ms/step
Accuracy: 0.9547844374342797
Precision: 0.2549019607843137
Recall: 0.7222222222222222
F1 Score: 0.3768115942028985
AUC-ROC: 0.840746695248303
Fold 4/5
30/30 [==============================] - 0s 1ms/step
Accuracy: 0.9379600420609885
Precision: 0.171875
Recall: 0.6470588235294118
F1 Score: 0.2716049382716049
AUC-ROC: 0.7951568207582819
Fold 5/5
30/30 [==============================] - 0s 2ms/step
Accuracy: 0.9526813880126183
Precision: 0.22
Recall: 0.6470588235294118
F1 Score: 0.3283582089552239
AUC-ROC: 0.802651467439224


In [19]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler

# STRAtified k-fold를 위한 설정
k_folds = 5  # k를 설정합니다.
stratified_kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

feature =x_train
label = y_train

# 교차 검증 수행
accuracies = []  # 각 fold의 정확도를 저장할 리스트

for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(feature, label)):
    print(f"Fold {fold+1}/{k_folds}")

    # Fold에 따라 데이터 분할
    X_train_fold, X_val_fold = feature.iloc[train_idx], feature.iloc[val_idx]
    y_train_fold, y_val_fold = label.iloc[train_idx], label.iloc[val_idx]

    # undersampling 수행
    rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train_fold, y_train_fold)

    X_train_resampled = X_train_resampled.values.reshape(X_train_resampled.shape[0], X_train_resampled.shape[1], 1)
    y_train_resampled = y_train_resampled.values

    # 모델 생성
    model = Sequential()
    model.add(Conv1D(128, 3, activation='relu', input_shape=(X_train_resampled.shape[1:])))
    model.add(MaxPooling1D(3))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 학습
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=64, verbose=0)  # verbose=0으로 설정하여 학습 과정 출력 생략

    # 검증 데이터에 대한 예측 및 평가
    X_val_fold = X_val_fold.values.reshape(X_val_fold.shape[0], X_val_fold.shape[1], 1)
    y_pred = model.predict(X_val_fold)
    predictions_binary = (y_pred > 0.3).astype(int)
    evaluate(y_val_fold, predictions_binary)

Fold 1/5
30/30 [==============================] - 0s 1ms/step
Accuracy: 0.9495798319327731
Precision: 0.2222222222222222
Recall: 0.6666666666666666
F1 Score: 0.3333333333333333
AUC-ROC: 0.8108493932905068
Fold 2/5
30/30 [==============================] - 0s 1ms/step
Accuracy: 0.9590336134453782
Precision: 0.2558139534883721
Recall: 0.6111111111111112
F1 Score: 0.36065573770491804
AUC-ROC: 0.7884249345705447
Fold 3/5
30/30 [==============================] - 0s 1ms/step
Accuracy: 0.9453207150368034
Precision: 0.22580645161290322
Recall: 0.7777777777777778
F1 Score: 0.35000000000000003
AUC-ROC: 0.8631654162200785
Fold 4/5
30/30 [==============================] - 0s 1ms/step
Accuracy: 0.9453207150368034
Precision: 0.2033898305084746
Recall: 0.7058823529411765
F1 Score: 0.3157894736842105
AUC-ROC: 0.8277805768988538
Fold 5/5
30/30 [==============================] - 0s 1ms/step
Accuracy: 0.9474237644584648
Precision: 0.18867924528301888
Recall: 0.5882352941176471
F1 Score: 0.2857142857142857

In [23]:
!pip install pytorch_tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 793.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [30]:
from pytorch_tabnet.tab_model import TabNetClassifier
from imblearn.under_sampling import RandomUnderSampler
import torch

X= x_train
y=y_train

rus = RandomUnderSampler(sampling_strategy=0.1)
X_resampled, y_resampled = rus.fit_resample(X, y)

X = X_resampled.values
y = y_resampled.values

# x_test, y_test = label_split(test_s)

X_test = x_test.values
y_test = y_test.values

# STRAtified k-fold를 위한 설정
k_folds = 5  # k를 설정합니다.
stratified_kfold = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# 교차 검증 수행
accuracies = []  # 각 fold의 정확도를 저장할 리스트

for fold, (train_idx, val_idx) in enumerate(stratified_kfold.split(X, y)):
    print(f"Fold {fold+1}/{k_folds}")

    # Fold에 따라 데이터 분할
    X_train_fold, X_val_fold = X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # 모델 생성
    clf = TabNetClassifier(
        n_d=8, n_a=8,
        optimizer_fn=torch.optim.Adam,
        scheduler_fn=torch.optim.lr_scheduler.CosineAnnealingLR,
        scheduler_params={'T_max': 100}
    )

    # 모델 학습
    clf.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        eval_metric=['accuracy'],
        max_epochs=100,  # 예시로 100으로 설정
        patience=20,  # 조기 종료를 위한 설정
        batch_size=1024,  # 배치 사이즈 설정
        virtual_batch_size=128,  # 가상 배치 사이즈 설정
        num_workers=0,  # 사용할 워커 수 설정
        drop_last=False
    )

    # 검증 데이터에 대한 예측
    y_pred = clf.predict(X_test)
    evaluate(y_test, y_pred)

Fold 1/5
epoch 0  | loss: 0.71819 | val_0_accuracy: 0.90722 |  0:00:00s
epoch 1  | loss: 0.54313 | val_0_accuracy: 0.90722 |  0:00:00s
epoch 2  | loss: 0.49767 | val_0_accuracy: 0.90722 |  0:00:00s
epoch 3  | loss: 0.44669 | val_0_accuracy: 0.90722 |  0:00:00s
epoch 4  | loss: 0.42445 | val_0_accuracy: 0.90722 |  0:00:00s
epoch 5  | loss: 0.38073 | val_0_accuracy: 0.90206 |  0:00:00s
epoch 6  | loss: 0.35468 | val_0_accuracy: 0.90722 |  0:00:00s
epoch 7  | loss: 0.30872 | val_0_accuracy: 0.91237 |  0:00:00s
epoch 8  | loss: 0.29861 | val_0_accuracy: 0.91753 |  0:00:00s
epoch 9  | loss: 0.28868 | val_0_accuracy: 0.90722 |  0:00:00s
epoch 10 | loss: 0.26271 | val_0_accuracy: 0.91237 |  0:00:00s
epoch 11 | loss: 0.26601 | val_0_accuracy: 0.91237 |  0:00:00s
epoch 12 | loss: 0.26166 | val_0_accuracy: 0.91237 |  0:00:00s
epoch 13 | loss: 0.23557 | val_0_accuracy: 0.92268 |  0:00:00s
epoch 14 | loss: 0.24203 | val_0_accuracy: 0.92268 |  0:00:00s
epoch 15 | loss: 0.22378 | val_0_accuracy: 0.9